In [5]:
import os
os.chdir('/code/libs/internvideo/InternVideo2/multi_modality')
import cv2

from demo_config import (Config,
                    eval_dict_leaf)

from demo.utils import (retrieve_text,
                  _frame_from_video,
                  setup_internvideo2)


video = cv2.VideoCapture('demo/example1.mp4')
frames = [x for x in _frame_from_video(video)]  # 40 x [480, 640, 3]

text_candidates = ["A playful dog and its owner wrestle in the snowy yard, chasing each other with joyous abandon.",
                   "A man in a gray coat walks through the snowy landscape, pulling a sleigh loaded with toys.",
                   "A person dressed in a blue jacket shovels the snow-covered pavement outside their house.",
                   "A pet dog excitedly runs through the snowy yard, chasing a toy thrown by its owner.",
                   "A person stands on the snowy floor, pushing a sled loaded with blankets, preparing for a fun-filled ride.",
                   "A man in a gray hat and coat walks through the snowy yard, carefully navigating around the trees.",
                   "A playful dog slides down a snowy hill, wagging its tail with delight.",
                   "A person in a blue jacket walks their pet on a leash, enjoying a peaceful winter walk among the trees.",
                   "A man in a gray sweater plays fetch with his dog in the snowy yard, throwing a toy and watching it run.",
                   "A person bundled up in a blanket walks through the snowy landscape, enjoying the serene winter scenery."]

config = Config.from_file('demo/internvideo2_stage2_config.py')
config = eval_dict_leaf(config)

intern_model, tokenizer = setup_internvideo2(config)

texts, probs = retrieve_text(frames, text_candidates, model=intern_model, topk=5, config=config)

for t, p in zip(texts, probs):
    print(f'text: {t} ~ prob: {p:.4f}')


/home/hglee/.conda/envs/iv2/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/hglee/.conda/envs/iv2/lib/python3.10/site-packages/flash_attn/ops/fused_dense.py:30: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(
/home/hglee/.conda/envs/iv2/lib/python3.10/site-packages/flash_attn/ops/fused_dense.py:71: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output, *args):
/home/hglee/.conda/envs/iv2/lib/python3.10/site-packages/flash_attn/ops/fused_dense.py:252: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(

[2025-04-14 19:40:52,939] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/hglee/.conda/envs/iv2/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/hglee/.conda/envs/iv2/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/hglee/.conda/envs/iv2/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/hglee/.conda/envs/iv2/lib/python3.10/site-packages/torch/utils/checkpoint.py:92: UserWarning: No

text: A person dressed in a blue jacket shovels the snow-covered pavement outside their house. ~ prob: 0.4319
text: A playful dog slides down a snowy hill, wagging its tail with delight. ~ prob: 0.3569
text: A man in a gray hat and coat walks through the snowy yard, carefully navigating around the trees. ~ prob: 0.0925
text: A person in a blue jacket walks their pet on a leash, enjoying a peaceful winter walk among the trees. ~ prob: 0.0788
text: A man in a gray coat walks through the snowy landscape, pulling a sleigh loaded with toys. ~ prob: 0.0163


In [44]:
import os
os.chdir('/code/libs/internvideo/InternVideo2/multi_modality')
import torch
import numpy as np
from decord import VideoReader

from torchvision import transforms
from torchvision.transforms._transforms_video import NormalizeVideo, CenterCropVideo
from pytorchvideo import transforms as pv_transforms

video_transform = transforms.Compose(
    [
        pv_transforms.ShortSideScale(256),
        pv_transforms.Div255(),
        CenterCropVideo(224),
        NormalizeVideo(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ]
)

vr = VideoReader('demo/example1.mp4')
num_frames = len(vr)
frames = torch.from_numpy(vr.get_batch(np.arange(0, num_frames, num_frames // 4)[:4]).asnumpy()).float()
frames = video_transform(frames.permute(3, 0, 1, 2))
frames.shape

torch.Size([3, 4, 224, 224])

In [45]:
vid_feat = intern_model.get_vid_feat(frames.permute(1, 0, 2, 3)[None].cuda())
vid_feat.shape

/home/hglee/.conda/envs/iv2/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/hglee/.conda/envs/iv2/lib/python3.10/site-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


torch.Size([1, 512])

In [46]:
txt_feat = intern_model.get_txt_feat(text_candidates)
txt_feat.shape

torch.Size([10, 512])

In [47]:
with np.printoptions(suppress=True, precision=5, linewidth=200):
    print((100 * vid_feat @ txt_feat.T).softmax(dim=-1).cpu().numpy())

[[0.01341 0.00977 0.26432 0.00598 0.00018 0.06495 0.53769 0.09439 0.00479 0.00451]]


In [48]:
top_probs, top_labels = (100 * vid_feat @ txt_feat.T).softmax(dim=-1).cpu().topk(5, dim=-1)
for prob, label in zip(top_probs[0], top_labels[0]):
    print(f'text: {text_candidates[label]} ~ prob: {prob:.4f}')

text: A playful dog slides down a snowy hill, wagging its tail with delight. ~ prob: 0.5377
text: A person dressed in a blue jacket shovels the snow-covered pavement outside their house. ~ prob: 0.2643
text: A person in a blue jacket walks their pet on a leash, enjoying a peaceful winter walk among the trees. ~ prob: 0.0944
text: A man in a gray hat and coat walks through the snowy yard, carefully navigating around the trees. ~ prob: 0.0650
text: A playful dog and its owner wrestle in the snowy yard, chasing each other with joyous abandon. ~ prob: 0.0134
